# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

# Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [7]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [8]:
from pyspark.sql import SparkSession
session = SparkSession.builder.getOrCreate()
session

In [9]:
session.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

#### Passing other options to spark session:
    
    

In [10]:
session = SparkSession.builder.config('someoption.key', 'somevalue').getOrCreate()

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [11]:
import random

In [12]:
random.choice(['a','b','c'])
random.seed(42)

ids =range(12)

positions =[random.choice(['mechanic', 'sales', 'manager']) for id_ in ids]
positions

['manager',
 'mechanic',
 'mechanic',
 'manager',
 'sales',
 'mechanic',
 'mechanic',
 'mechanic',
 'manager',
 'mechanic',
 'manager',
 'manager']

In [13]:
#crea el dataframe con zip, rollo cremallera
rows = zip(ids, positions)
df = session.createDataFrame(rows)
df

DataFrame[_1: bigint, _2: string]

In [14]:
#coge las primeras 5 filas 
df.take(5)

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mechanic'),
 Row(_1=2, _2='mechanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales')]

In [15]:
#muestra las 5 primeras filas
df.show(5)

+---+--------+
| _1|      _2|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [16]:
from pyspark.sql import Row
#help(Row)

In [17]:
rows = [Row(id=id_,position=position_) for id_,position_ in zip(ids,positions)]
rows

[Row(id=0, position='manager'),
 Row(id=1, position='mechanic'),
 Row(id=2, position='mechanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales'),
 Row(id=5, position='mechanic'),
 Row(id=6, position='mechanic'),
 Row(id=7, position='mechanic'),
 Row(id=8, position='manager'),
 Row(id=9, position='mechanic'),
 Row(id=10, position='manager'),
 Row(id=11, position='manager')]

In [18]:
df = session.createDataFrame(rows)
df.show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [19]:
#otra forma es crear un schema
session.createDataFrame(zip(ids,positions), schema=['id', 'position'])

DataFrame[id: bigint, position: string]

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [20]:
rdd = session.sparkContext.textFile('./coupon150720.csv')
rdd

./coupon150720.csv MapPartitionsRDD[22] at textFile at NativeMethodAccessorImpl.java:0

In [21]:
rdd.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [22]:
split_rdd = rdd.map(lambda line: line.split(','))
split_rdd.take(2)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0']]

In [23]:
new_df = session.createDataFrame(split_rdd)
new_df

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

### Inferring and specifying schemas

In [24]:
session.createDataFrame(zip(ids,positions), schema=['id', 'position'])

DataFrame[id: bigint, position: string]

In [25]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [26]:
from pyspark.sql import types

In [27]:
types.IntegerType()

IntegerType

In [28]:
fields = [types.StructField('id', types.IntegerType()), 
          types.StructField('position', types.StringType())]

my_schema = types.StructType(fields)

In [29]:
session.createDataFrame(zip(ids,positions), schema=my_schema)

DataFrame[id: int, position: string]

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [30]:
df_from_csv = session.read.csv("./coupon150720.csv")
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [31]:
#Se puede hacer de otra forma distinta....
df_with_sql = session.sql('SELECT * FROM csv.`./coupon150720.csv`')
df_with_sql.show(2)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W|56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W|84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 2 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [32]:
session.read.json

<bound method DataFrameReader.json of <pyspark.sql.readwriter.DataFrameReader object at 0x7fe709ef8898>>

### Basic operations with DataFrames

In [33]:
df.take(2)

[Row(id=0, position='manager'), Row(id=1, position='mechanic')]

In [34]:
df.schema

StructType(List(StructField(id,LongType,true),StructField(position,StringType,true)))

In [35]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



### Filtering and selecting

Syntax inspired in SQL.

In [36]:
df.select('id')

DataFrame[id: bigint]

In [37]:
filtered = df.filter(df['id']> 5)
filtered

DataFrame[id: bigint, position: string]

In [38]:
filtered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [39]:
df.where(df['position']== 'manager').select('id').show()


+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



`where` is exactly synonimous with `filter`

#### Exercise

Extract all employee ids which correspond to managers

In [40]:
whered = df.where(df['id']>5)

### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we not assing to them. We need to create new RDDS with the appropriate columns.

In [41]:
df2= df.withColumn('anewcolumn', df['id']* 100)
df2

DataFrame[id: bigint, position: string, anewcolumn: bigint]

In [42]:
#sin nombre
df3 = df.select('id', 'position', df['id']*100)
df3.show(3)

+---+--------+----------+
| id|position|(id * 100)|
+---+--------+----------+
|  0| manager|         0|
|  1|mechanic|       100|
|  2|mechanic|       200|
+---+--------+----------+
only showing top 3 rows



### User defined functions

Creating `udf`s allows us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [51]:
from pyspark.sql import functions
import math

In [52]:
functions.sqrt(df['id'])

Column<b'SQRT(id)'>

If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [53]:
#se le puede pasar tb. sólo el nombre de la columna
df.select('id', 'position', functions.sqrt(df['id'])).show(3)

+---+--------+------------------+
| id|position|          SQRT(id)|
+---+--------+------------------+
|  0| manager|               0.0|
|  1|mechanic|               1.0|
|  2|mechanic|1.4142135623730951|
+---+--------+------------------+
only showing top 3 rows



In [54]:
#Estas funciones udf funcionan como las lambda para este tipo de casos.
udf_log1p = functions.udf(math.log1p)


In [55]:
udf_log1p(col)

NameError: name 'col' is not defined

In [56]:
df.select(odd_udf('position'))

NameError: name 'odd_udf' is not defined

Think about this function: what is its return type?

#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [80]:
def salary (p):
    if p =='mechanic': 
        return 30000
    elif p =='sales': 
        return 40000
    elif p == 'manager': 
        return 70000


salary_udf = functions.udf(salary)
salary_udf


<function __main__.salary>

In [81]:
df.select('id', 'position', salary_udf(df['position'])).show(5)

+---+--------+----------------+
| id|position|salary(position)|
+---+--------+----------------+
|  0| manager|           70000|
|  1|mechanic|           30000|
|  2|mechanic|           30000|
|  3| manager|           70000|
|  4|   sales|           40000|
+---+--------+----------------+
only showing top 5 rows



In [83]:
#otra forma con diccionarios, utiliza la función get para devolver cero si le pasas una position q no está.
def salary_from (position):
        salaries = {'manager': 70000, 'sales': 40000, 'mechanic': 30000}
        return salaries.get(position,0)

salary_from('manager')
salary_udf = functions.udf(salary_from)

In [90]:
exercise_result = df.select('id', 'position', salary_udf('position').alias('salary').cast(types.IntegerType()))
exercise_result.show(5)

+---+--------+------+
| id|position|salary|
+---+--------+------+
|  0| manager| 70000|
|  1|mechanic| 30000|
|  2|mechanic| 30000|
|  3| manager| 70000|
|  4|   sales| 40000|
+---+--------+------+
only showing top 5 rows



In [87]:
exercise_result.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- salary_from(position): string (nullable = true)



### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [94]:
exercise_result.stat.corr('id', 'salary')

0.20742413513940736

In [95]:
exercise_result.stat.cov('id', 'salary')

14999.999999999995

### .crosstab()

In [105]:
location_udf = functions.udf(lambda : random.choice(['Madrid', 'Barcelona']))

In [114]:
#El cache es para reutilizar el dataframe varias veces y lo estabiliza con los datos.
df4 = exercise_result.withColumn('location', location_udf()).cache()

In [115]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|   Madrid|
|  2|mechanic| 30000|   Madrid|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|   Madrid|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|   Madrid|
|  8| manager| 70000|   Madrid|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|   Madrid|
| 11| manager| 70000|   Madrid|
+---+--------+------+---------+



In [117]:
#Tabla de concurrencia entre dos variables.
df4.crosstab('position', 'location').show(4)

+-----------------+---------+------+
|position_location|Barcelona|Madrid|
+-----------------+---------+------+
|            sales|        1|     0|
|          manager|        2|     3|
|         mechanic|        2|     4|
+-----------------+---------+------+



### Grouping

In [118]:
df4.groupby('location')

In [119]:
#Convertirlo en pandas
df4.toPandas().groupby('location')

In [121]:
df4.groupBy('location').mean().show()

+---------+-----------------+------------------+
| location|          avg(id)|       avg(salary)|
+---------+-----------------+------------------+
|   Madrid|6.285714285714286|47142.857142857145|
|Barcelona|              4.4|           48000.0|
+---------+-----------------+------------------+



In [127]:
stats = df4.groupby('location').agg(functions.mean('salary').alias('average'), functions.stddev_pop('salary').alias('stddev'))

stats.show()

+---------+------------------+-----------------+
| location|           average|           stddev|
+---------+------------------+-----------------+
|   Madrid|47142.857142857145|19794.86637221574|
|Barcelona|           48000.0|18330.30277982336|
+---------+------------------+-----------------+



### Intersections

Ver much like SQL joins.

In [140]:
data = list(zip([10,11,12,10,12],
           [5000,10000,2000,0,1000],
           [random.choice(['Madrid','Barcelona','Sevilla']) for _ in range(5)]))


In [149]:
new_df = session.createDataFrame(data, schema=['id', 'bonus', 'location'])

In [152]:
with_bonuses = df4.join(new_df, on=['id','location'], how='left')

In [153]:
type(with_bonuses)

pyspark.sql.dataframe.DataFrame

In [156]:
#Para acceder al Job Spark
sc.uiWebUrl

'http://10.0.2.15:4040'

In [157]:
df4.storageLevel

StorageLevel(True, True, False, True, 1)

In [158]:
from pyspark.storagelevel import StorageLevel

df4.persist()

DataFrame[id: bigint, position: string, salary: int, location: string]

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

In [183]:
stats = df4.groupby('location').agg(functions.mean('salary').alias('average'), functions.stddev_pop('salary').alias('stddev'))


Or we can use the built-in functions defined in [the `pyspark.sql` module](http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.functions)

Once calculated, we annotate each employee with the values for their location and calculate their z-score

In [189]:
#Realizamos una join de los dos dataframes por el campo location, left join...
#En el on, se pueden meter condiciones booleanas....
df_agg = df4.join(stats, on=['location'], how='left')
#df_agg.show(10)

#Lo hago con el método withColumn.....
df_with_Zscore = df_agg.withColumn('z-score', (df_agg['salary']-df_agg['average']/df_agg['stddev']))
#df_with_Zscore.show(10)

#otra forma es con select.......
result = df_agg.select('id','position','location',(df_agg['salary']-df_agg['average']/df_agg['stddev']).alias('z-score'))
result.show(10)


+---+--------+---------+------------------+
| id|position| location|           z-score|
+---+--------+---------+------------------+
|  0| manager|Barcelona| 69997.38138531717|
|  1|mechanic|   Madrid| 29997.61843013959|
|  2|mechanic|   Madrid| 29997.61843013959|
|  3| manager|Barcelona| 69997.38138531717|
|  4|   sales|Barcelona| 39997.38138531717|
|  5|mechanic|   Madrid| 29997.61843013959|
|  6|mechanic|Barcelona|29997.381385317167|
|  7|mechanic|   Madrid| 29997.61843013959|
|  8| manager|   Madrid|  69997.6184301396|
|  9|mechanic|Barcelona|29997.381385317167|
+---+--------+---------+------------------+
only showing top 10 rows



In [190]:
result.printSchema

<bound method DataFrame.printSchema of DataFrame[id: bigint, position: string, location: string, z-score: double]>

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

In [194]:
the_nulls = session.createDataFrame([[10,'manager',120000,None],[102,None, 150000,'Haiti']])

the_nulls.show()

+---+-------+------+-----+
| _1|     _2|    _3|   _4|
+---+-------+------+-----+
| 10|manager|120000| null|
|102|   null|150000|Haiti|
+---+-------+------+-----+



In [198]:
df5 =df4.union(the_nulls)

In [199]:
df5.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|   Madrid|
|  2|mechanic| 30000|   Madrid|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|   Madrid|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|   Madrid|
|  8| manager| 70000|   Madrid|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|   Madrid|
| 11| manager| 70000|   Madrid|
| 10| manager|120000|     null|
|102|    null|150000|    Haiti|
+---+--------+------+---------+



In [202]:
#borrar los na, no mostrarlos,
df5.dropna(thresh=3)

#Rellenar los na
df5.fillna('unknown').show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|   Madrid|
|  2|mechanic| 30000|   Madrid|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
|  5|mechanic| 30000|   Madrid|
|  6|mechanic| 30000|Barcelona|
|  7|mechanic| 30000|   Madrid|
|  8| manager| 70000|   Madrid|
|  9|mechanic| 30000|Barcelona|
| 10| manager| 70000|   Madrid|
| 11| manager| 70000|   Madrid|
| 10| manager|120000|  unknown|
|102| unknown|150000|    Haiti|
+---+--------+------+---------+



## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

In [213]:
df5.registerTempTable('df5_table')
session.sql('SELECT * FROM df5_table WHERE LOCATION IS NOT NULL').show(5)

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|   Madrid|
|  2|mechanic| 30000|   Madrid|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|Barcelona|
+---+--------+------+---------+
only showing top 5 rows



Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

In [218]:
#Convirtiendo a Pandas...
pd_df=df5.toPandas()
#en la otra dirección
session.createDataFrame(pd_df)

DataFrame[id: bigint, position: string, salary: bigint, location: string]

## Writing out


In [229]:
#Escribe un directorio en local, en distribuido escribe cada trozo en cada nodo dentro del hdfs
df5.write.csv('df5')

AnalysisException: 'path file:/home/dsc/MASTER/SPARK/df5 already exists.;'

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

In [228]:
raw_coupouns = session.read.csv('./coupon150720.csv')
raw_coupouns.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [242]:
coupons= raw_coupouns.select(raw_coupouns['_c0'].alias('ticket_number').cast(types.IntegerType(),
                             raw_coupouns['_c1'].alias('cpn_number').cast(types.IntegerType(),                                      
                             raw_coupouns['_c2'].alias('origin').cast(types.IntegerType(),                                      
                             raw_coupouns['_c3'].alias('destination').cast(types.IntegerType(),                                      
                             raw_coupouns['_c4'].alias('airline').cast(types.IntegerType(),                                      
                             raw_coupouns['_c6'].alias('amount').cast(types.IntegerType()
                            )
coupons.show(5)
coupons.printSchema


SyntaxError: invalid syntax (<ipython-input-242-19900e436bc2>, line 8)

In [ ]:
raw_coupouns.registerTempTable('raw_coupouns_table')
session.sql(''' SELECT 
            CAST(_c0 AS LONG) AS tkt_number
            CAST(_c1 AS INT) AS cpn_number,
            _c2 AS origin,
            _c3 AS LONG AS destination,
            CAST(_c4 AS LONG) AS tkt_number,
            CAST(_c6 AS FLOAT) AS tkt_number
            FROM raw_coupouns_table 
            WHERE DESTI="MAD" """).show(5)

## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

